<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/DB_09_model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **== Functions and setups ==**

## Setups

In [ ]:
!pip install torchinfo

In [7]:
import os
import requests
from pathlib import Path
import zipfile

import matplotlib.pyplot as plt
from tqdm.auto import tqdm


import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import summary

## Functions

In [ ]:
# train -> train_loss, train_acc
def train(model, dataloader, loss_fn, optimizer, device):
    model.to(device)
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X_train, y_train) in enumerate(dataloader):
        X_train, y_train = X_train.to(device), y_train.to(device)

        train_preds = model(X_train)

        loss = loss_fn(train_preds, y_train)
        train_loss = train_loss + loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_preds_label = torch.argmax(torch.softmax(train_preds, dim = 1), dim = 1)
        train_acc = train_acc + (train_preds_label == y_train).sum().item() / len(train_preds)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

In [4]:
# test -> test_loss, test_acc
def test(model, dataloader, loss_fn, device):
    model.to(device)
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X_test, y_test) in enumerate(dataloader):
            X_test, y_test = X_test.to(device), y_test.to(device)

            test_preds = model(X_test)

            loss = loss_fn(test_preds, y_test)
            test_loss = test_loss + loss.item()

            test_preds_label = torch.argmax(torch.softmax(test_preds, dim = 1), dim = 1)
            test_acc = test_acc + (test_preds_label == y_test).sum().item() / len(test_preds)

        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)

        return test_loss, test_acc

In [16]:
# train_test_loop -> results dictionary
def train_test_loop(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs, device):
    results = {"train_loss":[], "train_acc":[], "test_loss":[], "test_acc":[]}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train(model = model, dataloader = train_dataloader, loss_fn = loss_fn, optimizer = optimizer, device = device)
        test_loss, test_acc = test(model = model, dataloader = test_dataloader, loss_fn = loss_fn, device = device)

        print(f"Epoch: {epoch+1}\n"
              f"Train Loss: {train_loss:.4f} | Train Accuracy: {(train_acc*100):.2f}%\n"
              f"Test Loss: {test_loss:.4f} | Train Accuracy: {(test_acc*100):.2f}%")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

    # plt.figure(figsize = (6, 3))
    # plt.subplot(1, 2, 1)
    # plt.plot(results["train_loss"], label = "Train Loss")
    # plt.plot(results["test_loss"], label = "Test Loss")
    # plt.subplot(1, 2, 2)
    # plt.plot(results["train_acc"], label = "Train Accuracy")
    # plt.plot(results["test_acc"], label = "Test Accuracy")

# **== Dataset and dataloader ==**

## Raw data

In [13]:
data_dir = Path("data/")
image_dir = data_dir / "pizza_steak_sushi"

if image_dir.is_dir():
    print(f"[INFO] {image_dir} already exists")
else:
    image_dir.mkdir(parents = True, exist_ok = True)
    print(f"[INFO] {image_dir} creating")

    with open(data_dir / "DB_image.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print(f"[INFO] zip file downloading")
        f.write(request.content)

    with zipfile.ZipFile(data_dir / "DB_image.zip", "r") as zip_f:
        print(f"[INFO] zip file unzipping")
        zip_f.extractall(image_dir)

    zip_file_path = data_dir / "DB_image.zip"
    if zip_file_path.exists():
        print(f"[INFO] zip file removing")
        os.remove(zip_file_path)

[INFO] data/pizza_steak_sushi creating
[INFO] zip file downloading
[INFO] zip file unzipping


## Dataloader

In [21]:
# 目標比較: efficientnet VS vision transformer
train_dir = image_dir / "train"
test_dir = image_dir / "test"

# Weights
effnet_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnet_transforms = effnet_weights.transforms()

# Dataset
effnet_train_dataset = datasets.ImageFolder(root = train_dir, transform = effnet_transforms)
effnet_test_dataset = datasets.ImageFolder(root = test_dir, transform = effnet_transforms)

# Dataloader
effnet_train_dataloader = DataLoader(dataset = effnet_train_dataset, batch_size = 32, shuffle = True, drop_last = True)
effnet_test_dataloader = DataLoader(dataset = effnet_test_dataset, batch_size = 32, shuffle = False, drop_last = True)

In [22]:
# Weights
vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
vit_tranforms = vit_weights.transforms()

# Dataset
vit_train_dataset = datasets.ImageFolder(root = train_dir, transform = vit_tranforms)
vit_test_dataset = datasets.ImageFolder(root = test_dir, transform = vit_tranforms)

# Dataloader
vit_train_dataloader = DataLoader(dataset = vit_train_dataset, batch_size = 32, shuffle = True, drop_last = True)
vit_test_dataloader = DataLoader(dataset = vit_test_dataset, batch_size = 32, shuffle = False, drop_last = True)

# Model and train model

## Effnet

In [31]:
effnet_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

effnet_model = torchvision.models.efficientnet_b2(weights = effnet_weights)

In [32]:
for param in effnet_model.parameters():
    param.requires_grad = False

In [33]:
effnet_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p = 0.3),
    torch.nn.Linear(in_features = 1408, out_features = 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.3),
    torch.nn.Linear(in_features = 256, out_features = 64),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.3),
    torch.nn.Linear(in_features = 64, out_features = 3)
)

In [42]:
summary(effnet_model,
        input_size = (32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width = 17)

Layer (type:depth-idx)                                  Input Shape       Output Shape      Param #           Trainable
EfficientNet                                            [32, 3, 224, 224] [32, 3]           --                Partial
├─Sequential: 1-1                                       [32, 3, 224, 224] [32, 1408, 7, 7]  --                False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224] [32, 32, 112, 112] --                False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224] [32, 32, 112, 112] (864)             False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112] [32, 32, 112, 112] (64)              False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112] [32, 32, 112, 112] --                --
│    └─Sequential: 2-2                                  [32, 32, 112, 112] [32, 16, 112, 112] --                False
│    │    └─MBConv: 3-4                                 [32, 

## Vit

In [24]:
vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

vit_model = torchvision.models.vit_b_16(weights = vit_weights)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 107MB/s] 


In [35]:
for param in vit_model.parameters():
    param.requires_grad = False

In [43]:
vit_model.heads = torch.nn.Sequential(
    torch.nn.Linear(in_features=768, out_features=128),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.3),
    torch.nn.Linear(in_features = 128, out_features = 32),
    torch.nn.ReLU(),
    torch.nn.Dropout(p = 0.3),
    torch.nn.Linear(in_features = 32, out_features = 3)
)

In [44]:
summary(vit_model,
        input_size = (32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width = 17)

Layer (type:depth-idx)                        Input Shape       Output Shape      Param #           Trainable
VisionTransformer                             [32, 3, 224, 224] [32, 3]           768               Partial
├─Conv2d: 1-1                                 [32, 3, 224, 224] [32, 768, 14, 14] (590,592)         False
├─Encoder: 1-2                                [32, 197, 768]    [32, 197, 768]    151,296           False
│    └─Dropout: 2-1                           [32, 197, 768]    [32, 197, 768]    --                --
│    └─Sequential: 2-2                        [32, 197, 768]    [32, 197, 768]    --                False
│    │    └─EncoderBlock: 3-1                 [32, 197, 768]    [32, 197, 768]    (7,087,872)       False
│    │    └─EncoderBlock: 3-2                 [32, 197, 768]    [32, 197, 768]    (7,087,872)       False
│    │    └─EncoderBlock: 3-3                 [32, 197, 768]    [32, 197, 768]    (7,087,872)       False
│    │    └─EncoderBlock: 3-4              

# temp